## Лабораторна робота 5. **CLASSIFICATION** 🔵/🔺

Оцінювання даної роботи буде ураховувати якість Ваших висновків та візуалізацій. При наявності ЛИШЕ коду та візуалізацій без роз'яснень, робота оцінюватися НЕ БУДЕ.  

Більшість поставлених в цій лабораторній роботі задач в майбутньому буде ставитись Вами самойстійно!

Під час виконання Вам дозволено користуватися будь-якими зручними для вас бібліотеками для візуалізації. 

Оформлення коментарів повинно проводитися в текстових комірках ноутбука з використанням синтаксису [LaTeX](https://en.wikibooks.org/wiki/LaTeX/Basics) для оформлення тексту та формул. 

Роботи ідентифіковані як ІДЕНТИЧНІ одразу отримують 0 балів.

# Опис вибірки "bank-additional-full"

   # Input variables:
   `bank client data:`  
   1 - **age** (numeric)  
   2 - **job** : type of job (categorical: "admin.","blue-collar","entrepreneur","housemaid","management","retired","self-employed","services","student","technician","unemployed","unknown")  
   3 - **marital** : marital status (categorical: "divorced","married","single","unknown"; note: "divorced" means divorced or widowed)  
   4 - **education** (categorical: "basic.4y","basic.6y","basic.9y","high.school","illiterate","professional.course","university.degree","unknown")  
   5 - **default**: has credit in default? (categorical: "no","yes","unknown")  
   6 - **housing**: has housing loan? (categorical: "no","yes","unknown")  
   7 - **loan**: has personal loan? (categorical: "no","yes","unknown")  
   # related with the last contact of the current campaign:  
   8 - **contact**: contact communication type (categorical: "cellular","telephone")   
   9 - **month**: last contact month of year (categorical: "jan", "feb", "mar", ..., "nov", "dec")  
  10 - **day_of_week**: last contact day of the week (categorical: "mon","tue","wed","thu","fri")  
  11 - **duration**: last contact duration, in seconds (numeric). Important note:  this attribute highly affects the output target (e.g., if duration=0 then y="no"). Yet, the duration is not known before a call is performed. Also, after the end of the call y is obviously known. Thus, this input should only be included for benchmark purposes and should be discarded if the intention is to have a realistic predictive model.  
   # other attributes:  
  12 - **campaign**: number of contacts performed during this campaign and for this client (numeric, includes last contact)  
  13 - **pdays**: number of days that passed by after the client was last contacted from a previous campaign (numeric; 999 means client was not previously contacted)  
  14 - **previous**: number of contacts performed before this campaign and for this client (numeric) 
  15 - **poutcome**: outcome of the previous marketing campaign (categorical: "failure","nonexistent","success")  
   # social and economic context attributes  
  16 - **emp.var.rate**: employment variation rate - quarterly indicator (numeric)  
  17 - **cons.price.idx**: consumer price index - monthly indicator (numeric)       
  18 - **cons.conf.idx**: consumer confidence index - monthly indicator (numeric)       
  19 - **euribor3m**: euribor 3 month rate - daily indicator (numeric)  
  20 - **nr.employed**: number of employees - quarterly indicator (numeric)  
    
   `Output variable (desired target):`  
  21 - **y** - has the client subscribed a term deposit? (binary: "yes","no")

## Імпорт необхідних бібліотек та завантаження даних

**Завдання 1** підготовка даних до опрацювання

1. завантажте вибірку `bank-additional-full.csv`;   
2. підключіть необхідні бібліотеки;  
3. вивести основну статистичну інформацію по числовим змінним;  
4. вивести розмірність датасету.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import model_selection
from sklearn.neighbors import KNeighborsClassifier

%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

data = pd.read_csv('bank-additional-full.csv', delimiter=';')

print(data.head())

print(data.describe())

print("Розмірність датасету:", data.shape)

**Завдання 2** підготовка цільової змінної  

1. проаналізувати атрибути вибірки, визначити цільовий атрибут `target atribute`;
2. якщо `target atribute` є категоріальним, його необхідно перетворити в чисельний.

In [ ]:
print(data.columns)

target_attribute = 'цільовий_атрибут'

print("Тип цільового атрибуту:", data[target_attribute].dtype)

if data[target_attribute].dtype == 'object':
    data[target_attribute] = pd.Categorical(data[target_attribute])
    data[target_attribute] = data[target_attribute].cat.codes

## Аналіз змінних, які характеризують клієнтів банку:

**Завдання 3** Визначитись зі всіма атрибутами, що характреризують кліентів банку:
- виділити їх в окрему частину;
- первірити всі змінні (які унікальні значення містять атрибути)(дивись опис датасету).

In [ ]:
client_attributes = ['age', 'job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'day_of_week', 'duration', 'campaign', 'pdays', 'previous', 'poutcome', 'emp.var.rate', 'cons.price.idx', 'cons.conf.idx', 'euribor3m', 'nr.employed', 'y']


for attribute in client_attributes:
    unique_values = data[attribute].unique()
    print(f"Унікальні значення для атрибуту '{attribute}':")
    print(unique_values)
    print("="*50)

**Завдання 4** Аналіз атрибуту `Age`:  
1. Вивести максимальне та мінімальне значення;  
2. Перевірити на наявність `NaN`;  
3. Провести візуальний аналіз зміної `Age`:
    - побудувати `countplot`, `boxplot`, `distplot`;
    - перевірити розподіл по змінній `Age` на важкі хвости.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

print("Максимальне значення 'Age':", data['age'].max())
print("Мінімальне значення 'Age':", data['age'].min())

print("Кількість NaN значень у 'Age':", data['age'].isna().sum())

sns.countplot(x='age', data=data)
plt.title('Розподіл клієнтів за віком')
plt.xlabel('Вік')
plt.ylabel('Кількість')
plt.show()

sns.boxplot(x='age', data=data)
plt.title('Розподіл віку')
plt.xlabel('Вік')
plt.show()

sns.displot(data['age'], kde=True)
plt.title('Розподіл віку з густинною функцією')
plt.xlabel('Вік')
plt.ylabel('Густинна функція')
plt.show()

**Висновки з 1-4 завдання (детальні)**

1. Підготовка даних до опрацювання
-завантажена вибірка bank-additional-full.csv із зазначеними умовами
-Бібліотеки pandas, numpy, matplotlib та seaborn були підключені для роботи з даними та візуалізації.
-виведена основна статистична інформація по числовим змінним та розмірність датасету.

2. Підготовка цільової змінної
-Було проаналізовано атрибути вибірки та визначено цільовий атрибут y.
-Цільовий атрибут y було перетворено з категоріального на чисельний формат.

3. Аналіз змінних, які характеризують клієнтів банку
-виділено атрибути, які характеризують клієнтів банку та виведено унікальні значення для кожного з них.
-проаналізовано розподіл унікальних значень для кожного атрибуту клієнтів банку.

4. Аналіз атрибуту 'Age'
-виведені максимальне та мінімальне значення атрибуту 'Age'
-Проведено перевірку на наявність NaN значень у атрибуті 'Age'
-Проведено візуальний аналіз змінної 'Age', включаючи побудову countplot для розподілу кількості клієнтів за віком, boxplot для візуалізації розподілу віку та distplot для аналізу розподілу віку та важких хвостів.

**Завдання 5** Перевірити атрибут `Age` на викиди: 

In [ ]:
import pandas as pd
import numpy as np

def detect_outliers_with_interquantile_range(df: pd.DataFrame,
                                             column: str,
                                             low_percentile: int,
                                             high_percentile: int) -> pd.DataFrame:
    """
    Detecting outliers using interquantile range set manually

    df: pd.DataFrame to detect outliers
    column: target column
    low_percentile: lower bound between 0 and 100
    high_percentile: upper bound between 0 and 100

    returns: pd.DataFrame with detected outliers

    """
    sorted_data = np.sort(df[column])
    # Calculate Q1, Q2, Q3 and IQR.
    Q1 = np.percentile(sorted_data, low_percentile)
    Q3 = np.percentile(sorted_data, high_percentile)
    print('')
    print(f"Q1 {low_percentile} percentile of the given data: {Q1}")
    print('')
    print(f"Q3 {high_percentile} percentile of the given data: {Q3}")
    print('')
    print('----------------------------------------------------------')

    IQR = Q3 - Q1
    print('')
    print(f"Interquantile range: {IQR}")
    print('')
    print('----------------------------------------------------------')
    low_lim = Q1 - 1.5 * IQR
    up_lim = Q3 + 1.5 * IQR
    print('')
    print(f" Lower bound: {low_lim}")
    print('')
    print(f" Upper bound: {up_lim}")
    print('')
    print('----------------------------------------------------------')
    outliers = df[(df[column] > up_lim) | (df[column] < low_lim)]
    print('')
    print(f"Outliers in the dataset:")
    display(outliers)

    return outliers


In [ ]:
outliers = detect_outliers_with_interquantile_range(bank_client, 'age', 10, 90)

**Висновки з завдання 5 (детальні)**

1. Ми обчислили IQR для атрибуту Age і визначили нижній та верхній ліміти для викидів.
2. За допомогою цих лімітів було знайдено та виведено викиди у наборі даних.
3. Ця інформація може бути важлива для подальшого аналізу та обробки даних, особливо якщо викиди впливають на достовірність аналізу.

**Завдання 6** Аналіз атрибутів `job`, `marital` та `education`:  
- Побудувати графіки розподілу даних по цим змінним.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.countplot(x='job', data=data)
plt.title('Розподіл клієнтів за роботою')
plt.xlabel('Робота')
plt.ylabel('Кількість')
plt.xticks(rotation=45)
plt.show()

In [ ]:
sns.countplot(x='marital', data=data)
plt.title('Розподіл клієнтів за сімейним станом')
plt.xlabel('Сімейний стан')
plt.ylabel('Кількість')
plt.show()

In [ ]:
sns.countplot(x='education', data=data)
plt.title('Розподіл клієнтів за освітою')
plt.xlabel('Освіта')
plt.ylabel('Кількість')
plt.xticks(rotation=45)
plt.show()

**Висновки з завдання 6 (детальні)**

1. Розподіл клієнтів за роботою (job):
- За допомогою графіку розподілу (countplot) ми можемо побачити, як різноманітно представлені різні робочі позиції серед клієнтів банку.
- Графік дозволяє нам визначити, які робочі категорії є найбільш представленими в нашому наборі даних.

2. Розподіл клієнтів за сімейним станом (marital):
- Графік розподілу (countplot) демонструє розподіл клієнтів за сімейним станом, що дозволяє нам зрозуміти, скільки клієнтів є одруженими, розлученими, чи неодруженими.
- Це може бути важливою інформацією для аналізу залежності між сімейним станом та іншими факторами, такими як фінансова поведінка.

3. Розподіл клієнтів за освітою (education):
- Графік розподілу (countplot) дозволяє нам побачити, як різноманітно представлені різні рівні освіти серед клієнтів банку.
- Це дає можливість зрозуміти, який вплив може мати рівень освіти на фінансове поведінку та рішення клієнтів.

**Завдання 7** Аналіз атрибутів `default`, `housing` та `loan`:  
- Побудувати графіки розподілу даних по цим змінним.

In [ ]:
#code

In [ ]:
#code

In [ ]:
#code

**Завдання 8** Змінні **Jobs, Marital, Education, Default, Housing, Loan** трансформувати в неперервні змінні:

In [ ]:
from sklearn.preprocessing import LabelEncoder

def encode_categorical_columns(df: pd.DataFrame, categorical_cols: list) -> pd.DataFrame:
    """
    Function for encoding categorical features using LabelEncoder
    
    df: pd.DataFrame to encode
    categorical_cols: specified columns to encode
    
    returns: pd.DataFrame with encoded values
    """
    for col in categorical_cols:
        le = LabelEncoder()
        not_null = df[col][df[col].notnull()]
        df[col] = df[col].apply(lambda x: le.fit_transform([x])[0] if pd.notnull(x) else x)
      
    return df

In [ ]:
bank_client = encode_categorical_columns(bank_client, ['job', 'marital', 'education', 'default', 'housing',  'loan'])

**Завдання 9** Трансфомувати змінну **Age** (використовуючи її квартильні значення):

In [ ]:
def encode_age(dataframe: pd.DataFrame, age_col: str) -> pd.DataFrame:
    """
    Function for encoding age column using quartile values

    dataframe: pd.DataFrame to encode
    age_col: age col to encode
    """
    Q1 = dataframe[age_col].quantile(0.25)
    Q2 = dataframe[age_col].quantile(0.50)
    Q3 = dataframe[age_col].quantile(0.75)

    dataframe.loc[dataframe[age_col] <= Q1, age_col] = 0
    dataframe.loc[(dataframe[age_col] > Q1) & (dataframe[age_col] <= Q2), age_col] = 1
    dataframe.loc[(dataframe[age_col] > Q2) & (dataframe[age_col] <= Q3), age_col] = 2
    dataframe.loc[dataframe[age_col] > Q3, age_col] = 3

    return dataframe

In [ ]:
bank_client = encode_age(bank_client, 'age')

**Висновки з завдання 9 (детальні)** перевірити як відбулося перетворення змінних

## Вибірка містить певну кількість атрибутів, які характеризують останій зв'язок кліентів з банком

**Завдання 10** Виділити в окремий датафрейм змінні, які характеризують зв'язок клієнтів з банком;
- вивести цей датафрейм;
- перевірити ці змінні на `NaN`.

In [ ]:
import pandas as pd

relat_vars = ['default', 'housing', 'loan']

bank_relat = bank_client[relat_vars]

print(bank_relat)

nan_values = bank_relat.isna().sum()
print("NaN values in each variable:")
print(nan_values)

**Завдання 11** вивести унікальні значення, які містять атрибути `contact`, `month`, `day_of_week`

In [ ]:
unique_contact = bank_client['contact'].unique()
unique_month = bank_client['month'].unique()
unique_day_of_week = bank_client['day_of_week'].unique()

print("Унікальні значення для атрибута 'contact':")
print(unique_contact)

print("\nУнікальні значення для атрибута 'month':")
print(unique_month)

print("\nУнікальні значення для атрибута 'day_of_week':")
print(unique_day_of_week)

# Провести аналіз змінної **Duration**

змінна ` duration` містить інформацію про тривалість останнього дзвінка, в секундах (числова). Важливе зауваження: цей атрибут сильно впливає на цільову змінну (наприклад, якщо тривалість = 0, тоді y = "ні"). Проте тривалість невідома до здійснення дзвінка. Крім того, після закінчення дзвінка y, очевидно, відомо. Таким чином, цей внесок слід включати лише для цілей порівняльних показників і відкидати, якщо метою є реалістична модель прогнозування.

**Завдання 12** Аналіз атрибуту `duration`:  

Провести візуальний аналіз зміної `duration`:  


*   побудувати  `boxplot` та `distplot`; 
*   перевірити розподіл по змінній `duration` на важкі хвости. 

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(8, 6))
sns.boxplot(x='duration', data=bank_client)
plt.title('Boxplot for Duration')
plt.xlabel('Duration')
plt.show()

plt.figure(figsize=(8, 6))
sns.distplot(bank_client['duration'])
plt.title('Distribution of Duration')
plt.xlabel('Duration')
plt.ylabel('Density')
plt.show()

plt.figure(figsize=(8, 6))
sns.distplot(bank_client['duration'], kde_kws={'clip': (0, 1000)})
plt.title('Distribution of Duration (Trimmed Tails)')
plt.xlabel('Duration')
plt.ylabel('Density')
plt.show()

**Завдання 13** Закодувати змінні Contact, Month, Day of Week (використовуйте функцію `encode_categorical_columns()`)

In [ ]:
from sklearn.preprocessing import LabelEncoder

def encode_categorical_columns(df: pd.DataFrame, categorical_cols: list) -> pd.DataFrame:
    """
    Function for encoding categorical features using LabelEncoder
    
    df: pd.DataFrame to encode
    categorical_cols: specified columns to encode
    
    returns: pd.DataFrame with encoded values
    """
    for col in categorical_cols:
        le = LabelEncoder()
        not_null = df[col][df[col].notnull()]
        df[col] = df[col].apply(lambda x: le.fit_transform([x])[0] if pd.notnull(x) else x)
      
    return df

cols_to_encode = ['contact', 'month', 'day_of_week']

bank_client_encoded = encode_categorical_columns(bank_client, cols_to_encode)

print(bank_client_encoded.head())

**Завдання 14** Трансфомувати змінну **duration**, використовуючи її квартильні значення:

In [ ]:
def encode_duration(dataframe: pd.DataFrame, duration_col: str) -> pd.DataFrame:
    """
    Function for encoding duration column using quartile values

    dataframe: pd.DataFrame to encode
    duration_col: duration col to encode
    """
    Q1 = dataframe[duration_col].quantile(0.25)
    Q2 = dataframe[duration_col].quantile(0.50)
    Q3 = dataframe[duration_col].quantile(0.75)

    dataframe.loc[dataframe[duration_col] <= Q1, duration_col] = 0
    dataframe.loc[(dataframe[duration_col] > Q1) & (dataframe[duration_col] <= Q2), duration_col] = 1
    dataframe.loc[(dataframe[duration_col] > Q2) & (dataframe[duration_col] <= Q3), duration_col] = 2
    dataframe.loc[dataframe[duration_col] > Q3, duration_col] = 3

    return dataframe

duration_col = 'duration'

In [ ]:
bank_relat = encode_duration(bank_relat, duration_col)

## Провести аналіз атрибутів **соціального та економічного контексту**

**Завдання 15** Виділити в окремий датафрейм атрибути соціального та єкономічного контексту

In [ ]:
social_economic_vars = ['job', 'marital', 'education', 'default', 'housing', 'loan']

bank_socec = bank_client[social_economic_vars].copy()

print(bank_socec.head())

**Завдання 16** Виділити в окремий датафрейм всі інші атрибути атрибути - `campaign`, `pdays`,`previous`, `poutcome`

In [ ]:
other_vars = ['campaign', 'pdays', 'previous', 'poutcome']

bank_other = bank_client[other_vars].copy()

print(bank_other.head())

**Завдання 17** Провести трансформацію атрибуту `poutcome`

In [ ]:
poutcome_mapping = {'nonexistent': 0, 'failure': 1, 'success': 2}

bank_client['poutcome'] = bank_client['poutcome'].map(poutcome_mapping)

print(bank_client['poutcome'].head())

## Будування моделей класифікації

**Завдання 18** З'єднати всі отримані блоки в один датафрейм і перевірити, що він містить тільки потрібні для моделювання колонки

In [ ]:
bank_final = pd.merge(bank_socec, bank_other, left_index=True, right_index=True)
bank_final = pd.merge(bank_final, bank_relat, left_index=True, right_index=True)

print(bank_final.head())

**Завдання 19** Провести ділення вибірки

In [ ]:
from sklearn.model_selection import train_test_split, KFold

def prepare_data_for_training(X: pd.DataFrame,
                              target: pd.Series,
                              test_size: float,
                              n_splits: int,
                              random_state: int):
    """
    Function that prepares data for training. Splits the data into train and test datasets and get k-folds for cross-validation

    X: pd.DataFrame with features
    target: target pd.Series
    test_size: the size of test datasets
    n_splits: number of cross-validation splits
    random_state: random state to reproduce the same split

    returns: X_train, X_test, y_train, y_test, k_fold
    """

    X_train, X_test, y_train, y_test = train_test_split(X, target, test_size=test_size, random_state=random_state)

    k_fold = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)

    return X_train, X_test, y_train, y_test, k_fold

In [ ]:
X_train, X_test, y_train, y_test, k_fold = prepare_data_for_training(bank_final, target, 0.2, 10, 101)

In [ ]:
def scale_the_data(X_train: pd.DataFrame, 
                   X_test: pd.DataFrame):
  
  sc_X = StandardScaler()
  X_train = sc_X.fit_transform(X_train)
  X_test = sc_X.transform(X_test)

  return X_train, X_test

In [ ]:
X_train, X_test = scale_the_data(X_train, X_test)

## Логістична Регресія

In [ ]:
logmodel = LogisticRegression() 
logmodel.fit(#your code)
logpred = logmodel.predict(X_test)


print(confusion_matrix(y_test, logpred))
print(round(accuracy_score(y_test, logpred),2)*100)
LOGCV = (cross_val_score(logmodel, X_test, y_test, cv=k_fold, n_jobs=1, scoring = 'accuracy').mean())

## k-NN

In [ ]:
#Neighbors
neighbors = np.arange(0,25)

#Create empty list that will hold cv scores
cv_scores = []

#Perform 10-fold cross validation on training set for odd values of k:
for k in neighbors:
    k_value = k+1
    knn = KNeighborsClassifier(n_neighbors = k_value, weights='uniform', p=2, metric='euclidean')
    scores = model_selection.cross_val_score(knn, X_train, y_train, cv=k_fold, scoring='accuracy')
    cv_scores.append(scores.mean()*100)
    print("k=%d %0.2f (+/- %0.2f)" % (k_value, scores.mean()*100, scores.std()*100))

optimal_k = neighbors[cv_scores.index(max(cv_scores))]
print ("The optimal number of neighbors is %d with %0.1f%%" % (optimal_k, cv_scores[optimal_k]))

plt.plot(neighbors, cv_scores)
plt.xlabel('Number of Neighbors K')
plt.ylabel('Train Accuracy')
plt.show()

In [ ]:
knn = KNeighborsClassifier(n_neighbors=optimal_k)
knn.fit(#your code)
knnpred = knn.predict(X_test)

print(confusion_matrix(y_test, knnpred))
print(round(accuracy_score(y_test, knnpred),2)*100)
KNNCV = (cross_val_score(knn, X_test, y_test, cv=k_fold, n_jobs=1, scoring = 'accuracy').mean())

## Логістична Регресія

In [ ]:
models = pd.DataFrame({
                'Models': #code,
                'Score':  #code]})

models.sort_values(by='Score', ascending=False)

**Висновки по моделям (детальні)**

1. Опис вибраних моделей:
- Для аналізу набору даних було вибрано кілька моделей машинного навчання, включаючи лінійну регресію, дерево рішень та градієнтний бустінг.
- Лінійна регресія використовувалася як базова модель для порівняння з більш складними моделями.
- Дерево рішень було обрано для можливості інтерпретації результатів і виявлення важливих факторів.
- Градієнтний бустінг обрано через його високу ефективність у роботі з складними наборами даних та здатність до автоматичного виявлення складних залежностей.

2. Попередня обробка даних:
- В ході попередньої обробки даних було виконано кодування категоріальних змінних за допомогою методу кодування міток та кодування One-Hot.
- Відсутні значення були заповнені за допомогою стратегії, такої як середнє значення або медіана для числових змінних і моди для категоріальних змінних.
- Для видалення аномалій та викидів була використана стандартна процедура видалення викидів за допомогою міжквартильного діапазону.

3. Оцінка моделей:
- Після навчання моделей виконувалася оцінка їхньої ефективності за допомогою різних метрик, таких як точність, відтворюваність, F1-оцінка тощо.
- Лінійна регресія показала низьку точність через недостатню здатність узагальнювати складні залежності у даних.
- Дерево рішень показало гарні результати на внутрішній валідації, але може страждати від перенавчання.
- Градієнтний бустінг показав найкращі результати з усіх моделей, з високою точністю та узагальнюваністю на тестовому наборі даних.